# Защита персональных данных клиентов

Требуется защитить данные клиентов страховой компании. Необходимо предложить и обосновать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

## Загрузка данных

### Импорт библиотек

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

import numpy as np
from matplotlib import pyplot as plt

import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade Pillow
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

from ydata_profiling import ProfileReport

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


### Первичный анализ данных

In [2]:
df = pd.read_csv('insurance.csv')

In [5]:
profile = ProfileReport(df, title="Insurance")
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
4990    0     22.0   56800.0            1                  0
4991    0     21.0   38300.0            3                  0
4992    0     45.0   54600.0            0                  1
4993    1     32.0   36000.0            1                  0
4994    1     26.0   51400.0            0                  0
4995    0     28.0   35700.0            2                  0
4996    0     34.0   52400.0            1                  0
4997    0     20.0   33900.0            2                  0
4998    1     22.0   32700.0            3                  0
4999    1     28.0   40600.0            1                  0

### Выводы по анализу данных

- датасет на удивление пристойный. Пропусков нет, дубликатов менее 3%. Дубликаты удалю.
- распределение зарпаты похоже на нормальное, что логично.
- распределение количества членов семьи понятное. Больших семей меньше. 
- распределение возраста также объяснимо. Видно плато наиболее активного периода жизни 25-40 лет. Пик 18-летних также не является аномалией, поскольку именно с наступлением совершеннолетия люди получают право оформить страховку, а значит, при одинаковых потребностях, 18-летних клиентов будет больше, чем 19-летних
- положительная корреляция между страховыми выплатами за период 5 лет и возрастом, вероятно, объясняется тем, что с возрастом обычно появляется больше имущества, которое имеет смысл страховать

In [6]:
df.duplicated().sum()

153

In [7]:
df[df.duplicated()].head()

Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
281    1     39.0   48100.0            1                  0
488    1     24.0   32900.0            1                  0
513    0     31.0   37400.0            2                  0
718    1     22.0   32600.0            1                  0
785    0     20.0   35800.0            0                  0

### Удаление дубликатов

In [8]:
df = df.drop_duplicates().reset_index(drop=True)

### Подготовка данных

In [9]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

## Защита данных

Предлагается для защиты персональных данных изменять матрицу признаков таким образом, чтобы затруднить интерпретацию записанных в ней данных

### Доказательство

При обучении вектор весов для признаков находятся по следующей формуле:

$$
w = (X^T X)^{-1} X^T y
$$

После чего решается задача минимизации среднего квадратичного отклонения между таргетом $y$ и произведением 

$$a=Xw$$

Обозначения:

- $X$ — матрица признаков

- $y$ — вектор целевого признака

- $w$ — вектор весов линейной регрессии

- $P$ — матрица, на которую будут умножаться признаки. По условию задачи (и, как видно из формулы для $w$) матрица должна быть обратимая. То есть матрица $P$ имеет обратную матрицу $P^{-1}$, такую что $P \cdot P^{-1} = E$, где  $E$ - единичная матрица. Это значит, что она квадратная размера ($X.shape[1] \times X.shape[1]$) (чтобы быть совместимой с матрицей $X$) и невырожденная ($detA \ne 0$)

- $c=X.shape[1]$ - число столбцов матрицы $X$

- $r=X.shape[0]$ - число строк матрицы $X$

Размерности матриц:

- $X - r \times c$
- $P - c \times c$
- $X^{T} - c \times r$
- $P^{T} - c \times c$

Свойства матриц, которые буду использовать:

- $(AB)^{T}=B^{T}A^{T}$ - транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке
- $(AB)^{-1}=B^{-1}A^{-1}$ - обратное произведение **обратимых(!)** матриц равно произведению обратных матриц, взятых в обратном порядке

**Докажем, что если умножить матрицу $X$ на квадратную обратимую матрицу $P$, то значение $a$ не изменится, то есть**

$X'w'=Xw$,

где 

$X'=XP$, 

$w'=(X'^{T}X')^{-1}X'^{T}y$

$w'=((XP)^{T}(XP))^{-1}(XP)^{T}y = (P^{T}X^{T}XP)^{-1}P^{T}X^{T}y = ((P^{T}) \cdot (X^{T}XP))^{-1}P^{T}X^{T}y$

Заметим, что матрицы $P^{T}$ и $X^{T}XP$ квадратные с размерностью $c \times c$, а значит можно воспользоваться свойством обратного произведения матриц:

$w' = (X^{T}XP)^{-1}(P^{T})^{-1}P^{T}X^{T}y = (X^{T}XP)^{-1}EX^{T}y = (X^{T}XP)^{-1}X^{T}y$

Матрицы $P$ и $X^{T}X$ также квадратные с размерностью $c \times c$, снова воспользуемся свойством обратного произведения матриц:

$w' = P^{-1}(X^{T}X)^{-1}X^{T}y$

Легко видеть, что

$X'w' = XPw' = XPP^{-1}(X^{T}X)^{-1}X^{T}y = XE(X^{T}X)^{-1}X^{T}y = X(X^{T}X)^{-1}X^{T}y = Xw$

**Что и требовалось доказать**

### Вывод

**Доказано, что для линейной регрессии умножение матрицы признаков на любую обратимую матрицу на результат предсказаний не влияет**

## Алгоритм преобразования

В задании сказано: "Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию". При этом, что такое "сложно", насколько сложно и т.д. не говорится. Поэтому в качестве алгоритмов преобразования будут рассмотрены:

- масштабирование признаков стандартным способом (как пример защиты просто от любопытного взора) :)
- умножение матрицы признаков на генерируемую по ключу случайную обратимую матрицу

### Алгоритм

- задается ключ - состояние генератора случайных чисел
- генерируется случайная двумерная квадратная матрица, совместимая с матрицей признаков, при этом в цикле добавлена проверка, что матрица невырожденная. Понятно, что вероятность получить вырожденную матрицу, мягко скажем, невысока, но подстраховаться надо. 

In [10]:
det_m = 0
state = 1983
while det_m == 0:
    state += 1
    KEY = np.random.default_rng(state)
    M = KEY.random((features.shape[1], features.shape[1]))
    det_m = np.linalg.det(M)
    
KEY = np.random.default_rng(state)
p = M
print(f'Ключ - состояние генератора случайных чисел - {state}')

Ключ - состояние генератора случайных чисел - 1984


In [11]:
KEY = np.random.default_rng(state)
m = KEY.integers(1, 100)
value = np.ones(features.shape[1])
p_diag = np.diagflat(value) * m
#print(p_diag)

### Обоснование

Как было показано выше, умножение матрицы признаков на обратнимую матрицу правомерно. А запоминание состояния генератора случайных чисел позволит не только зашифровать, но и расшифровать данные

## Проверка алгоритма

**Проверка будет производиться методом расчета среднего значения метрики R2 после кросс-валидации.**

**Число блоков принято равным 5**

### Исходные признаки

In [12]:
print('MEAN R2_SCORE DEFAULT = ', round(cross_val_score(LinearRegression(), features, target, scoring='r2', cv=5).mean(),2))

MEAN R2_SCORE DEFAULT =  0.43


### Масштабированные признаки

In [13]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
print('MEAN R2_SCORE SCALED = ', round(cross_val_score(LinearRegression(), \
                                    features_scaled, target, scoring='r2', cv=5).mean(),2))

MEAN R2_SCORE SCALED =  0.43


### Зашифрованные признаки

Исходные признаки. Запомним их такими. 

In [14]:
features.head()

Пол  Возраст  Зарплата  Члены семьи
0    1     41.0   49600.0            1
1    0     46.0   38000.0            1
2    0     29.0   21000.0            0
3    0     21.0   41700.0            2
4    1     28.0   26100.0            0

In [15]:
features_pr = features.values @ p

In [16]:
pd.DataFrame(features_pr, columns=features.columns).head()

Пол       Возраст     Зарплата   Члены семьи
0  6264.808870  31899.698802  7653.347995  38516.349956
1  4807.040851  24447.436557  5873.870068  29511.838353
2  2657.832061  13512.414244  3248.538073  16309.487368
3  5261.076678  26811.388412  6424.100977  32379.653318
4  3299.460274  16789.514638  4032.231044  20268.676154

Так, зашифровать получилось. 

Попробуем расшифровать

In [17]:
KEY = np.random.default_rng(state)
p1 = np.linalg.inv(KEY.random((features.shape[1], features.shape[1])))
pd.DataFrame(features_pr @ p1, columns=features.columns).head()

Пол  Возраст  Зарплата   Члены семьи
0  1.000000e+00     41.0   49600.0  1.000000e+00
1 -1.364242e-12     46.0   38000.0  1.000000e+00
2  0.000000e+00     29.0   21000.0 -9.094947e-13
3  3.183231e-12     21.0   41700.0  2.000000e+00
4  1.000000e+00     28.0   26100.0 -1.818989e-12

Нууууу, данным типа int немного поплохело, что неудивительно. Но если округлить и вернуть им их родной тип, все вернется, как было

In [18]:
print('MEAN R2_SCORE PROTECTED = ', round(cross_val_score(LinearRegression(), \
                                            features_pr, target, scoring='r2', cv=5).mean(),2))

MEAN R2_SCORE PROTECTED =  0.43


### Зашифрованные с помощью диагональной или треугольной матрицы признаки

In [19]:
features_pr_diag = features.values @ p_diag

In [20]:
pd.DataFrame(features_pr_diag, columns=features.columns).head()

Пол  Возраст   Зарплата  Члены семьи
0  25.0   1025.0  1240000.0         25.0
1   0.0   1150.0   950000.0         25.0
2   0.0    725.0   525000.0          0.0
3   0.0    525.0  1042500.0         50.0
4  25.0    700.0   652500.0          0.0

Так себе зашифровал, честно говоря:)
    
Возьму лучше треугольную, умноженную на случайное число

In [21]:
KEY = np.random.default_rng(state)
m = KEY.integers(1, 100)
p_tri = np.tri(features.shape[1], dtype=int) * m
#print(p_tri)

In [22]:
features_pr_tri = features.values @ p_tri

In [23]:
pd.DataFrame(features_pr_tri, columns=features.columns).head()

Пол    Возраст   Зарплата  Члены семьи
0  1241075.0  1241050.0  1240025.0         25.0
1   951175.0   951175.0   950025.0         25.0
2   525725.0   525725.0   525000.0          0.0
3  1043075.0  1043075.0  1042550.0         50.0
4   653225.0   653200.0   652500.0          0.0

In [24]:
print('MEAN R2_SCORE PROTECTED = ', round(cross_val_score(LinearRegression(), \
                                            features_pr_tri, target, scoring='r2', cv=5).mean(),2))

MEAN R2_SCORE PROTECTED =  0.43


## Выводы по проекту

**В ходе работы было выполнено:**

- предварительный анализ и подготовка данных
- доказательство правомерности домножения матрицы признаков на любую обратимую матрицу для моделей линейной регрессии
- исследование и сравнение результатов кросс-валидации для модели линейной регресии для различных матриц признаков: исходной, отмасштабированной и зашифрованной (двумя способами)
- для всех вариантов получено значение R2_SCORE = 0,43 (Так себе результат, честно говоря). Зато одинаковый для всех. И это главное в данном случае